In [1]:
import pandas as pd
from sklearn.cross_validation import KFold, train_test_split
import xgboost
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.preprocessing import Normalizer
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
transactions = pd.read_csv('~/data/sberbank/transactions.csv')
sex = pd.read_csv('~/data/sberbank/customers_gender_train.csv')

sex = sex.set_index('customer_id')
transactions = transactions.set_index('customer_id')

In [3]:
def get_time(x):
    x = x.split(' ')[1]
    x = x.split(':')
    return int(x[0])*60 + int(x[1])
def regex(x):
    if pd.isnull(x):
        return np.nan
    x = re.findall('[a-zA-Z]+', x)
    if len(x) == 0:
        return ''
    return x[0]
def return_pairs(l):
    l = list(l)
    pairs = []
    for i in range(len(l) - 1):
        pairs.append((l[i], l[i + 1]))
    return pairs
def return_pairs2(l):
    l = list(l)
    pairs = []
    for i in range(len(l) - 2):
        pairs.append((l[i], l[i + 1], l[i + 2]))
    return pairs
def create_range(series):
    if len(series) < 2:
        return 0
    s = 0
    n = 0
    for i in range(len(series) - 1):
        s += series[i + 1] - series[i]
        n += 1
    return s / n
def prepeocessing(t, sex):
    
    users = t.index.unique()
    users2 = sex.index.values
    users_test = list(set.difference(set(users), users2))
    
    t['day'] = t.tr_datetime.apply(lambda x:int(x.split(' ')[0]))
    t['time'] = t.tr_datetime.apply(get_time)
    
    
    data = pd.DataFrame(index=t.index.unique())
    
    data['day_range'] = t.day.groupby(by=t.index).max() - t.day.groupby(by=t.index).min()
    data['day_strange'] = t.day.groupby(by=t.index).median() \
        - (t.day.groupby(by=t.index).max() + t.day.groupby(by=t.index).min()) / 2
    data['amount_min'] = t.amount.groupby(by=t.index).min()
    data['amount_max'] = t.amount.groupby(by=t.index).max()
    data['amount_mean'] = t.amount.groupby(by=t.index).mean()
    data['amount_mean2'] = t.amount[t.amount < 0].groupby(by=t.index[t.amount < 0]).mean()
    data['amount_median'] = t.amount.groupby(by=t.index).median()
    data['amount_median2'] = t.amount[t.amount < 0].groupby(by=t.index[t.amount < 0]).median()
    data['amount_sum'] = t.amount.groupby(by=t.index).sum()
    data['amount_sum2'] = t.amount[t.amount < 0].groupby(by=t.index[t.amount < 0]).sum()
    data['time_mean'] = t.time.groupby(by=t.index).mean()
    data['time_median'] = t.time.groupby(by=t.index).median()
    data['time_std'] = t.time.groupby(by=t.index).std()
    tmp = t[t.time != 0]
    data['time_std2'] = tmp.time.groupby(by=tmp.index).std()
    
    
    tmp = t.reset_index().groupby(['customer_id', 'day']).apply(len).reset_index()
    data['median_transaction_at_day'] = tmp[0].groupby(tmp.customer_id).median()
    
    tmp = pd.get_dummies(t.mcc_code, prefix='mcc_code').groupby(by=t.index).sum()
    data = data.join(tmp)
    
    tmp = pd.get_dummies(t.mcc_code, prefix='mcc_code__').groupby(by=t.index).mean()
    data = data.join(tmp)
    
    tmp = pd.get_dummies(t.tr_type, prefix='tr_type').groupby(by=t.index).sum()
    data = data.join(tmp)
    
    tmp = t.reset_index().groupby(['customer_id', 'mcc_code'])['amount'].mean().reset_index()
    tmp = tmp.pivot(index='customer_id', columns='mcc_code', values='amount').fillna(0)
    tmp.columns = [str(col) + '_amount' for col in tmp.columns]
    data = data.join(tmp)
    
    index = ((t.day > 217) & (t.day <= 219) | (t.day > 149) \
        & (t.day <= 152)) & (t.mcc_code == 5944)
    tmp = t[index]
    data['jewelry_8March_NewYear'] = tmp.amount.groupby(by=tmp.index).apply(len)
    
    index = (t.day > 218) & (t.day <= 219) & (t.mcc_code == 5992)
    tmp = t[index]
    data['flowers_8March'] = tmp.amount.groupby(by=tmp.index).apply(len)
    
    index = (t.day > 204) & (t.day <= 206) & (t.mcc_code == 5947)
    tmp2 = t[index]
    data['menswear_23Febrary'] = tmp.amount.groupby(by=tmp.index).apply(len)
    
    index = (t.day > 204) & (t.day <= 206) & (t.mcc_code == 5611)
    tmp2 = t[index]
    data['gift_23Febrary'] = tmp.amount.groupby(by=tmp.index).apply(len)
    
    tmp = t[t.amount < 0]
    codes = [5065,5072,5074,5094,5131,5511,5532,5533,5542,5571,5599,5611,
             5621,5631,5681,5681,5714,5734,5812,5945,5947,5949,5977,5993,7230]
    index = np.array([i in codes for i in tmp.mcc_code.values])
    tmp = tmp[index]
    d = {}
    index = tmp.index.values
    mcc = ['mcc_amount_' + str(i) for i in tmp.mcc_code.values]
    amount = tmp.amount.values
    for i in range(len(index)):
        d[index[i]] = {}
    for i in range(len(index)):
        if mcc[i] not in d[index[i]]:
            d[index[i]][mcc[i]] = amount[i]
        else:
            d[index[i]][mcc[i]] += amount[i]
    tmp = pd.DataFrame(d)
    tmp = tmp.transpose()
    tmp = tmp.fillna(0)
    tmp = -tmp
    norm = Normalizer(norm='l1')
    pca = PCA(n_components=3)
    result2 = pca.fit_transform(norm.fit_transform(tmp))
    data['pca1'] = np.nan
    data.loc[tmp.index, 'pca1'] = result2[:,0]
    
    tmp = t.mcc_code.groupby(by=t.index).apply(return_pairs)
    l = []
    for i in tmp.values:
        l += i
    result = Counter(l)
     
    for i in result.most_common()[:180]:
        index = [i[0] in pairs for pairs in tmp.values]
        data['Counter' + str(i[0])] = False
        data.loc[tmp[index].index, 'Counter' + str(i[0])] = True

    tmp = t[((t.day % 7 ==  1) | (t.day % 7  == 2)) & (t.amount < 0)]
    
    tmp = pd.get_dummies(tmp.mcc_code, prefix='mcc_code_weekday1').groupby(by=tmp.index).mean()
    data = data.join(tmp)
    
    tmp = t[t.amount < 0]
    tmp['weekday'] = abs(tmp.day % 7 - 1.5) < 1
    tmp2 = tmp[np.invert(tmp.weekday)].reset_index().groupby(['customer_id', 'mcc_code']).apply(len).reset_index()
    tmp3 = tmp[tmp.weekday].reset_index().groupby(['customer_id', 'mcc_code']).apply(len).reset_index()
    tmp2 = tmp2.pivot(index = 'customer_id', columns = 'mcc_code', values=0).fillna(0)
    tmp3 = tmp3.pivot(index = 'customer_id', columns = 'mcc_code', values=0).fillna(0)
    tmp2 = tmp2 + 1
    tmp3 = tmp3 + 1
    tmp = tmp2 / tmp3
    tmp.columns = [str(col) + 'weekday' for col in tmp.columns]
    data = data.join(tmp)
    
    tmp = t[t.amount < 0]
    tmp['weekday'] = abs(tmp.day % 7 - 1.5) < 1
    weekday = pd.Series(index=tmp.index.unique())
    weekday.loc[weekday.index] = 0
    tmp2 = tmp[np.invert(tmp.weekday)].groupby(tmp[np.invert(tmp.weekday)].index).apply(len)
    weekday.loc[tmp2.index] = tmp2
    weekday += 1
    weekend = pd.Series(index=tmp.index.unique())
    weekend.loc[weekend.index] = 0
    tmp3 = tmp[tmp.weekday].groupby(tmp[tmp.weekday].index).apply(len)
    weekend.loc[tmp3.index] = tmp3
    weekend += 1
    data['share_week'] = weekday / weekend
    
    tmp = t.reset_index().groupby(['customer_id', 'day'])['time'].apply(list)
    tmp2 = tmp.apply(np.std)
    tmp2 = tmp2[tmp2 != 0]
    tmp2 = tmp2.reset_index().groupby('customer_id')['time'].mean()
    data['std_time_over_day'] = tmp2
    tmp = t[t.time!= 0].reset_index().groupby(['customer_id', 'day'])['time'].apply(list)
    tmp2 = tmp.apply(create_range)
    tmp2 = tmp2[tmp2 != 0]
    tmp2 = tmp2.reset_index().groupby('customer_id')['time'].mean()
    data['range_time_over_day'] = tmp2

    X = data.loc[users2]
    y = sex.gender
    
    XTEST = data.loc[users_test]
    
    return X, y, XTEST


In [4]:
%%time
X, y, XTEST = prepeocessing(transactions, sex)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 6min 27s, sys: 27.9 s, total: 6min 55s
Wall time: 6min 56s


In [5]:
model1 = xgboost.XGBClassifier(n_estimators=1288, max_depth=1, seed=2, subsample=0.55, nthread=8)
model2 = xgboost.XGBClassifier(n_estimators=704,  max_depth=2, seed=2, subsample=0.7,  nthread=8)
model3 = xgboost.XGBClassifier(n_estimators=342,  max_depth=3, seed=2, subsample=0.7,  nthread=8)
model4 = xgboost.XGBClassifier(n_estimators=1288, max_depth=1, seed=1, subsample=0.55, nthread=8)
model5 = xgboost.XGBClassifier(n_estimators=704,  max_depth=2, seed=1, subsample=0.7,  nthread=8)
model6 = xgboost.XGBClassifier(n_estimators=342,  max_depth=3, seed=1, subsample=0.7,  nthread=8)
model7 = xgboost.XGBClassifier(n_estimators=216,  max_depth=4, seed=1, subsample=0.8,  nthread=8)
model8 = xgboost.XGBClassifier(n_estimators=235,  max_depth=5, seed=1, subsample=0.8,  nthread=8)
model9 = xgboost.XGBClassifier(n_estimators=286,  max_depth=6, seed=1, subsample=0.8,  nthread=8)
model10 = xgboost.XGBClassifier(n_estimators=450,  max_depth=7, seed=1, subsample=0.9,  nthread=8)
model11 = xgboost.XGBClassifier(n_estimators=450,  max_depth=8, seed=1, subsample=0.9,  nthread=8)

In [6]:
proba = [pd.Series(index=X.index) for i in range(9)]
models = [model1, model2, model3, model4, model5, model6, model7, model8, model9]
for train, test in KFold(len(X), n_folds=10, shuffle=True, random_state=241):
    for i in range(9):
        models[i].fit(X.iloc[train], y.iloc[train])
        proba[i].iloc[test] = models[i].predict_proba(X.iloc[test])[:,1]

In [7]:
proba.append(pd.Series(index=X.index))
proba.append(pd.Series(index=X.index))
for train, test in KFold(len(X), n_folds=10, shuffle=True, random_state=241):
    model10.fit(X.iloc[train], y.iloc[train])
    model11.fit(X.iloc[train], y.iloc[train])
    proba[9].iloc[test] = model10.predict_proba(X.iloc[test])[:,1]
    proba[10].iloc[test] = model11.predict_proba(X.iloc[test])[:,1]

In [8]:
def create_blending_coefficient(y, proba, max_iter=100, verbose=False):
    B = np.transpose(proba)
    n = len(proba)
    a = np.array([6]*n)
    roc_old = roc_auc_score(y, (B * a).mean(axis=1))
    for i in range(max_iter):
        a[i % n] += 1
        roc_new = roc_auc_score(y, (B * a).mean(axis=1))
        if roc_new < roc_old:
            a[i % n] += -2
            roc_new = roc_auc_score(y, (B * a).mean(axis=1))
            if roc_new <= roc_old:
                a[i % n] += 1
                roc_new = roc_old
        roc_old = roc_new
        if verbose:
            print(a, roc_old)
    a[a < 0] = 0
    print(roc_old, a)
    return a / a.sum()

In [9]:
a = create_blending_coefficient(y, proba, max_iter=100)

0.891763821104 [ 3 12  3  8 10 10  1  3  0  4  9]


In [10]:
probaTEST = []
for i in range(9):
    models[i].fit(X, y)
    probaTEST.append(models[i].predict_proba(XTEST)[:,1])

In [11]:
model10.fit(X, y)
model11.fit(X, y)
probaTEST.append(model10.predict_proba(XTEST)[:,1])
probaTEST.append(model11.predict_proba(XTEST)[:,1])

In [12]:
submit = pd.DataFrame(a.dot(probaTEST), XTEST.index, columns=['gender'])
submit = submit.sort_index().reset_index()
submit.columns = ['customer_id', 'gender']
submit.to_csv('A.csv', index=False)